# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [43]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. '''

## 2. Generate the mathematical model

In [44]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [45]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Chemical Product Production

**Parameters:**

* **Availability:**
    * A1 = 10,000 l (Available quantity of C1)
    * A2 = 15,000 l (Available quantity of C2)
* **Demand:**
    * D_GCA = 6,000 l (Minimum demand for GCA)
    * D_GCB = 7,000 l (Minimum demand for GCB)
    * D_GCC = 9,000 l (Minimum demand for GCC)
* **Critical Element Proportion:**
    * CE1 = 0.4 (Proportion of critical element in C1)
    * CE2 = 0.2 (Proportion of critical element in C2)
* **Critical Element Requirements:**
    * CE_GCA_min = 0.3 (Minimum proportion of critical element in GCA)
    * CE_GCB_max = 0.3 (Maximum proportion of critical element in GCB)
* **Component Ratio:**
    * CR_GCC = 0.3 (Minimum ratio of C1 to C2 in GCC)
* **Profit:**
    * P_GCA = $120/l (Profit per liter of GCA)
    * P_GCB = $135/l (Profit per liter of GCB)
    * P_GCC = $155/l (Profit per liter of GCC)

**Decision Variables:**

* **Production Quantities:**
    * X_GCA = Liters of GCA produced
    * X_GCB = Liters of GCB produced
    * X_GCC = Liters of GCC produced
* **Component Allocation:**
    * C1_GCA = Liters of C1 used in GCA
    * C2_GCA = Liters of C2 used in GCA
    * C1_GCB = Liters of C1 used in GCB
    * C2_GCB = Liters of C2 used in GCB
    * C1_GCC = Liters of C1 used in GCC
    * C2_GCC = Liters of C2 used in GCC 

**Objective Function:**

Maximize profit: 
```
Maximize Z = P_GCA * X_GCA + P_GCB * X_GCB + P_GCC * X_GCC
```

**Constraints:**

* **Demand Satisfaction:**
    * X_GCA >= D_GCA
    * X_GCB >= D_GCB
    * X_GCC >= D_GCC
* **Component Availability:**
    * C1_GCA + C1_GCB + C1_GCC <= A1
    * C2_GCA + C2_GCB + C2_GCC <= A2
* **Production Quantity:**
    * X_GCA = C1_GCA + C2_GCA
    * X_GCB = C1_GCB + C2_GCB
    * X_GCC = C1_GCC + C2_GCC 
* **Critical Element Requirements:**
    * (CE1 * C1_GCA + CE2 * C2_GCA) / X_GCA >= CE_GCA_min 
    * (CE1 * C1_GCB + CE2 * C2_GCB) / X_GCB <= CE_GCB_max
* **Component Ratio:**
    * C1_GCC / C2_GCC >= CR_GCC
* **Non-Negativity:**
    * All decision variables >= 0

This mathematical optimization model can be solved using linear programming techniques to determine the optimal production plan that maximizes the firm's profit while meeting demand, resource availability, and product composition requirements. 


## 3. Generate the pyomo code

In [46]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [47]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define parameters
model.A1 = pyo.Param(initialize=10000) # Available quantity of C1
model.A2 = pyo.Param(initialize=15000) # Available quantity of C2
model.D_GCA = pyo.Param(initialize=6000) # Minimum demand for GCA
model.D_GCB = pyo.Param(initialize=7000) # Minimum demand for GCB
model.D_GCC = pyo.Param(initialize=9000) # Minimum demand for GCC
model.CE1 = pyo.Param(initialize=0.4) # Proportion of critical element in C1
model.CE2 = pyo.Param(initialize=0.2) # Proportion of critical element in C2
model.CE_GCA_min = pyo.Param(initialize=0.3) # Minimum proportion of critical element in GCA
model.CE_GCB_max = pyo.Param(initialize=0.3) # Maximum proportion of critical element in GCB
model.CR_GCC = pyo.Param(initialize=0.3) # Minimum ratio of C1 to C2 in GCC
model.P_GCA = pyo.Param(initialize=120) # Profit per liter of GCA
model.P_GCB = pyo.Param(initialize=135) # Profit per liter of GCB
model.P_GCC = pyo.Param(initialize=155) # Profit per liter of GCC

# Define decision variables
model.X_GCA = pyo.Var(within=pyo.NonNegativeReals) # Liters of GCA produced
model.X_GCB = pyo.Var(within=pyo.NonNegativeReals) # Liters of GCB produced
model.X_GCC = pyo.Var(within=pyo.NonNegativeReals) # Liters of GCC produced
model.C1_GCA = pyo.Var(within=pyo.NonNegativeReals) # Liters of C1 used in GCA
model.C2_GCA = pyo.Var(within=pyo.NonNegativeReals) # Liters of C2 used in GCA
model.C1_GCB = pyo.Var(within=pyo.NonNegativeReals) # Liters of C1 used in GCB
model.C2_GCB = pyo.Var(within=pyo.NonNegativeReals) # Liters of C2 used in GCB
model.C1_GCC = pyo.Var(within=pyo.NonNegativeReals) # Liters of C1 used in GCC
model.C2_GCC = pyo.Var(within=pyo.NonNegativeReals) # Liters of C2 used in GCC

# Define objective function
def objective_rule(model):
  return model.P_GCA * model.X_GCA + model.P_GCB * model.X_GCB + model.P_GCC * model.X_GCC
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.demand_GCA = pyo.Constraint(expr=model.X_GCA >= model.D_GCA) # Demand satisfaction for GCA
model.demand_GCB = pyo.Constraint(expr=model.X_GCB >= model.D_GCB) # Demand satisfaction for GCB
model.demand_GCC = pyo.Constraint(expr=model.X_GCC >= model.D_GCC) # Demand satisfaction for GCC

model.availability_C1 = pyo.Constraint(expr=model.C1_GCA + model.C1_GCB + model.C1_GCC <= model.A1) # Component availability for C1
model.availability_C2 = pyo.Constraint(expr=model.C2_GCA + model.C2_GCB + model.C2_GCC <= model.A2) # Component availability for C2

model.production_GCA = pyo.Constraint(expr=model.X_GCA == model.C1_GCA + model.C2_GCA) # Production quantity for GCA
model.production_GCB = pyo.Constraint(expr=model.X_GCB == model.C1_GCB + model.C2_GCB) # Production quantity for GCB
model.production_GCC = pyo.Constraint(expr=model.X_GCC == model.C1_GCC + model.C2_GCC) # Production quantity for GCC

model.critical_element_GCA = pyo.Constraint(expr=(model.CE1 * model.C1_GCA + model.CE2 * model.C2_GCA) / model.X_GCA >= model.CE_GCA_min) # Critical element requirement for GCA
model.critical_element_GCB = pyo.Constraint(expr=(model.CE1 * model.C1_GCB + model.CE2 * model.C2_GCB) / model.X_GCB <= model.CE_GCB_max) # Critical element requirement for GCB

model.component_ratio_GCC = pyo.Constraint(expr=model.C1_GCC / model.C2_GCC >= model.CR_GCC) # Component ratio for GCC

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal Production Plan:")
print("X_GCA =", model.X_GCA())
print("X_GCB =", model.X_GCB())
print("X_GCC =", model.X_GCC())
print("C1_GCA =", model.C1_GCA())
print("C2_GCA =", model.C2_GCA())
print("C1_GCB =", model.C1_GCB())
print("C2_GCB =", model.C2_GCB())
print("C1_GCC =", model.C1_GCC())
print("C2_GCC =", model.C2_GCC())
print("Total Profit =", model.objective())
```

This code defines the parameters and decision variables, and then uses them to construct the objective function and constraints. The `pyo.SolverFactory('glpk')` line specifies that the GLPK solver will be used to solve the model. Finally, the code prints the optimal values of the decision variables and the total profit. 

This code uses the sample data provided in the problem statement. 


## 4. Input problem data and try running the generated code

In [49]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define parameters
model.A1 = pyo.Param(initialize=10000) # Available quantity of C1
model.A2 = pyo.Param(initialize=15000) # Available quantity of C2
model.D_GCA = pyo.Param(initialize=6000) # Minimum demand for GCA
model.D_GCB = pyo.Param(initialize=7000) # Minimum demand for GCB
model.D_GCC = pyo.Param(initialize=9000) # Minimum demand for GCC
model.CE1 = pyo.Param(initialize=0.4) # Proportion of critical element in C1
model.CE2 = pyo.Param(initialize=0.2) # Proportion of critical element in C2
model.CE_GCA_min = pyo.Param(initialize=0.3) # Minimum proportion of critical element in GCA
model.CE_GCB_max = pyo.Param(initialize=0.3) # Maximum proportion of critical element in GCB
model.CR_GCC = pyo.Param(initialize=0.3) # Minimum ratio of C1 to C2 in GCC
model.P_GCA = pyo.Param(initialize=120) # Profit per liter of GCA
model.P_GCB = pyo.Param(initialize=135) # Profit per liter of GCB
model.P_GCC = pyo.Param(initialize=155) # Profit per liter of GCC

# Define decision variables
model.X_GCA = pyo.Var(within=pyo.NonNegativeReals) # Liters of GCA produced
model.X_GCB = pyo.Var(within=pyo.NonNegativeReals) # Liters of GCB produced
model.X_GCC = pyo.Var(within=pyo.NonNegativeReals) # Liters of GCC produced
model.C1_GCA = pyo.Var(within=pyo.NonNegativeReals) # Liters of C1 used in GCA
model.C2_GCA = pyo.Var(within=pyo.NonNegativeReals) # Liters of C2 used in GCA
model.C1_GCB = pyo.Var(within=pyo.NonNegativeReals) # Liters of C1 used in GCB
model.C2_GCB = pyo.Var(within=pyo.NonNegativeReals) # Liters of C2 used in GCB
model.C1_GCC = pyo.Var(within=pyo.NonNegativeReals) # Liters of C1 used in GCC
model.C2_GCC = pyo.Var(within=pyo.NonNegativeReals) # Liters of C2 used in GCC

# Define objective function
def objective_rule(model):
  return model.P_GCA * model.X_GCA + model.P_GCB * model.X_GCB + model.P_GCC * model.X_GCC
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.demand_GCA = pyo.Constraint(expr=model.X_GCA >= model.D_GCA) # Demand satisfaction for GCA
model.demand_GCB = pyo.Constraint(expr=model.X_GCB >= model.D_GCB) # Demand satisfaction for GCB
model.demand_GCC = pyo.Constraint(expr=model.X_GCC >= model.D_GCC) # Demand satisfaction for GCC

model.availability_C1 = pyo.Constraint(expr=model.C1_GCA + model.C1_GCB + model.C1_GCC <= model.A1) # Component availability for C1
model.availability_C2 = pyo.Constraint(expr=model.C2_GCA + model.C2_GCB + model.C2_GCC <= model.A2) # Component availability for C2

model.production_GCA = pyo.Constraint(expr=model.X_GCA == model.C1_GCA + model.C2_GCA) # Production quantity for GCA
model.production_GCB = pyo.Constraint(expr=model.X_GCB == model.C1_GCB + model.C2_GCB) # Production quantity for GCB
model.production_GCC = pyo.Constraint(expr=model.X_GCC == model.C1_GCC + model.C2_GCC) # Production quantity for GCC

model.critical_element_GCA = pyo.Constraint(expr=(model.CE1 * model.C1_GCA + model.CE2 * model.C2_GCA) / model.X_GCA >= model.CE_GCA_min) # Critical element requirement for GCA
model.critical_element_GCB = pyo.Constraint(expr=(model.CE1 * model.C1_GCB + model.CE2 * model.C2_GCB) / model.X_GCB <= model.CE_GCB_max) # Critical element requirement for GCB

model.component_ratio_GCC = pyo.Constraint(expr=model.C1_GCC / model.C2_GCC >= model.CR_GCC) # Component ratio for GCC

# Solve the model
solver = pyo.SolverFactory('ipopt')
solver.solve(model)

# Print the results
print("Optimal Production Plan:")
print("X_GCA =", model.X_GCA())
print("X_GCB =", model.X_GCB())
print("X_GCC =", model.X_GCC())
print("C1_GCA =", model.C1_GCA())
print("C2_GCA =", model.C2_GCA())
print("C1_GCB =", model.C1_GCB())
print("C2_GCB =", model.C2_GCB())
print("C1_GCC =", model.C1_GCC())
print("C2_GCC =", model.C2_GCC())
print("Total Profit =", model.objective())


Optimal Production Plan:
X_GCA = 5999.999940000111
X_GCB = 6999.999930000195
X_GCC = 12000.000379999645
C1_GCA = 4092.7341651475467
C2_GCA = 1907.265774852564
C1_GCB = 850.631438956435
C2_GCB = 6149.36849104376
C1_GCC = 5056.634495895993
C2_GCC = 6943.365884103652
Total Profit = 3525000.042249984


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.

In [50]:
print(response.text)

## Mathematical Optimization Model for Chemical Product Production

**Parameters:**

* **Availability:**
    * A1 = 10,000 l (Available quantity of C1)
    * A2 = 15,000 l (Available quantity of C2)
* **Demand:**
    * D_GCA = 6,000 l (Minimum demand for GCA)
    * D_GCB = 7,000 l (Minimum demand for GCB)
    * D_GCC = 9,000 l (Minimum demand for GCC)
* **Critical Element Proportion:**
    * CE1 = 0.4 (Proportion of critical element in C1)
    * CE2 = 0.2 (Proportion of critical element in C2)
* **Critical Element Requirements:**
    * CE_GCA_min = 0.3 (Minimum proportion of critical element in GCA)
    * CE_GCB_max = 0.3 (Maximum proportion of critical element in GCB)
* **Component Ratio:**
    * CR_GCC = 0.3 (Minimum ratio of C1 to C2 in GCC)
* **Profit:**
    * P_GCA = $120/l (Profit per liter of GCA)
    * P_GCB = $135/l (Profit per liter of GCB)
    * P_GCC = $155/l (Profit per liter of GCC)

**Decision Variables:**

* **Production Quantities:**
    * X_GCA = Liters of GCA produce

In [51]:
print(response2.text)

```python
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define parameters
model.A1 = pyo.Param(initialize=10000) # Available quantity of C1
model.A2 = pyo.Param(initialize=15000) # Available quantity of C2
model.D_GCA = pyo.Param(initialize=6000) # Minimum demand for GCA
model.D_GCB = pyo.Param(initialize=7000) # Minimum demand for GCB
model.D_GCC = pyo.Param(initialize=9000) # Minimum demand for GCC
model.CE1 = pyo.Param(initialize=0.4) # Proportion of critical element in C1
model.CE2 = pyo.Param(initialize=0.2) # Proportion of critical element in C2
model.CE_GCA_min = pyo.Param(initialize=0.3) # Minimum proportion of critical element in GCA
model.CE_GCB_max = pyo.Param(initialize=0.3) # Maximum proportion of critical element in GCB
model.CR_GCC = pyo.Param(initialize=0.3) # Minimum ratio of C1 to C2 in GCC
model.P_GCA = pyo.Param(initialize=120) # Profit per liter of GCA
model.P_GCB = pyo.Param(initialize=135) # Profit per liter of GCB
model.P_GCC = pyo